For some reason CDN was being really annoying about returning a status code after I got the pdf url from Crossref. However, when I pasted that same link into my own browser, I downloaded the article just fine. Therefore, I'm using Selenium to automate the clicking for me.

I suddently started getting some 403 errors. I'll try again tomorrow.

In [149]:
import pandas as pd
from tqdm import tqdm
import json
import requests
from pathlib import Path
import re
import random
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from send2trash import send2trash
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [150]:
def download_pdf(doi, dest_folder, content):
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
    doi_suffix = re.sub(r"""([()/\\*,"': ?;<>])""", '_._', doi_suffix)
    filename = dest_folder + '/' + str(doi_suffix)+'.pdf'
    with open(filename, mode = 'wb') as f:
        f.write(content)

In [151]:
def get_publisher_download_url(crossref_resp, intended_app='text-mining'):
    # search for 'text-mining' in the intended-application field
    res = []
    for link in crossref_response.json()['link']:
        if re.search(intended_app,link['intended-application']):
            res.append(re.search(intended_app, link['intended-application']))
        else:
            res.append(None)
    # find the index of match obects in crossref link information
    publisher_api_idxs = np.where(np.array(res) != None)[0].tolist()
    # publisher_api_idxs
    links = [crossref_response.json()['link'][i] for i in publisher_api_idxs]
    # for now use the first link if there are more than one link
    full_text_link = links[0]['URL']
    return full_text_link

In [152]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]

In [ ]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
cdn_ca = articles[(articles['is_oa'] == False) & \
    (articles['publisher'] == 'Canadian Science Publishing')].drop_duplicates().reset_index(drop=True)

In [ ]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
cdn_ca['doi_suffix'] = cdn_ca['doi'].apply(lambda x: re_doi_suffix.search(x).group())
full_texts_to_download = [x for x in cdn_ca['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), cdn_ca, how='inner', on='doi_suffix')
merge.shape

(230, 50)

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/cdn_non_oa'

In [ ]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.implicitly_wait(1) # hopefully let's JS load correctly

In [ ]:
n = random.randint(0, merge.shape[0])
row = merge.loc[n,:]
doi = row['doi']
doi

'10.2466/pms.2002.95.3f.1035'

In [193]:
doi = row['doi']
out = {'doi': doi}
doi_url = 'https://doi.org/' + str(doi)
crossref_headers = {'User-Agent': user_agent, 'Accept': accept}
crossref_response = requests.get(
                url = doi_url,
                headers=crossref_headers,
                allow_redirects=True,
                verify=True)
full_text_url = get_publisher_download_url(crossref_response, intended_app='similarity-checking')
full_text_url

'http://www.nrcresearchpress.com/doi/pdf/10.1139/h2012-056'

In [192]:
crossref_response.json()['link']

[{'URL': 'http://www.nrcresearchpress.com/doi/full-xml/10.1139/h2012-056',
  'content-type': 'application/xml',
  'content-version': 'vor',
  'intended-application': 'text-mining'},
 {'URL': 'http://www.nrcresearchpress.com/doi/pdf/10.1139/h2012-056',
  'content-type': 'unspecified',
  'content-version': 'vor',
  'intended-application': 'similarity-checking'}]

In [ ]:
for idx, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    doi_url = 'https://doi.org/' + str(doi)
    crossref_headers = {'User-Agent': user_agent, 'Accept': accept}
    try:
        crossref_response = requests.get(
                        url = doi_url,
                        headers=crossref_headers,
                        allow_redirects=True,
                        verify=True)
        out.update({'CR_status_code': crossref_response.status_code})
        if crossref_response.status_code == 200:
            full_text_url = get_publisher_download_url(crossref_response, intended_app='similarity-checking')
            driver.get(full_text_url)
            time.sleep(3)
            full_text_resp = requests.get(url = driver.current_url, headers = {'User-Agent': user_agent})
            out.update({'full_text_SC': full_text_resp.status_code})
            if full_text_resp.status_code == 200:
                download_pdf(doi=doi, dest_folder=dest_folder, content=full_text_resp.content)
    except Exception as e:
        print(e)
        out.update({'error': e})
    log.append(out)

In [146]:
log_df = pd.DataFrame(log)
log_df

,doi,CR_status_code,full_text_SC,error
0,10.1139/apnm-2019-0187,200,200.0,NaN
1,10.1139/apnm-2019-0560,200,200.0,NaN
2,10.1139/apnm-2019-0405,200,200.0,NaN
3,10.1139/apnm-2019-0230,200,200.0,NaN
4,10.1139/apnm-2019-0037,200,200.0,NaN
...,...,...,...,...
337,10.1139/h97-028,200,403.0,NaN
338,10.1139/y93-143,200,403.0,NaN
339,10.1139/h08-031,200,403.0,NaN
340,10.1139/h00-004,200,403.0,NaN


In [187]:
log_df['full_text_SC'].value_counts()

403.0    240
200.0    100
Name: full_text_SC, dtype: int64

In [188]:
status_code_df = pd.merge(log_df, merge, how='inner', on='doi')
error_df = status_code_df[status_code_df['full_text_SC'] != 200].reset_index(drop=True)

In [182]:
status_code_df.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/cdn_non_oa_status_codes.csv',
                     index=False)

In [185]:
log_df.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/cdn_non_oa_log.csv',
                     index=False)

In [189]:
status_code_df.shape

(242, 53)